In [1]:
from anthropic import AnthropicBedrock
from application import Application
from compiler.core import Compiler
import tempfile

In [2]:
compiler = Compiler("botbuild/tsp_compiler", "botbuild/app_schema")
client = AnthropicBedrock(aws_profile="dev", aws_region="us-west-2")

In [3]:
tempdir = tempfile.TemporaryDirectory()
application = Application(client, compiler, "templates", tempdir.name)

In [4]:
application_description = """
Bot that tracks my exercise routine in the gym, tracks progress and suggests new routines
for specific list of available equipment and time constraints.
""".strip()

In [5]:
my_bot = application.create_bot(application_description)

Langfuse client is disabled since no public_key was provided as a parameter or environment variable 'LANGFUSE_PUBLIC_KEY'. See our docs: https://langfuse.com/docs/sdk/python/low-level-sdk#initialize-client


Compiling TypeSpec...
Generating Typescript Schema Definitions...
Compiling Drizzle...
Generating Router...
Generating Handlers...
Generating Application...


In [6]:
my_bot

{'typespec': TypespecData(messages=[{'role': 'assistant', 'content': '<reasoning>\nFor a greeting bot, I expect users to send messages like "Hello, I\'m John" or "Hi there, my name is Alice".\nThe LLM needs to extract the name from such messages. The main functionality would be:\n1. Storing the user\'s name when they introduce themselves\n2. Greeting them with their stored name\nBoth functions should be LLM-enabled as they need to process natural language input.\nThe setName function would extract names from introduction messages.\nThe greet function would generate appropriate greetings using the stored name.\n</reasoning>\n\n<typespec>\nmodel UserInfo {\n    name: string;\n}\n\ninterface GreetingBot {\n    @llm_func(1)\n    setName(user: UserInfo): void;\n\n    @llm_func(1)\n    greet(): string;\n}\n</typespec>'}], output=TypespecOutput(reasoning='For a greeting bot, I expect users to send messages like "Hello, I\'m John" or "Hi there, my name is Alice".\nThe LLM needs to extract the 

In [7]:
application.generation_dir

'/tmp/tmpl4gqx9ta/generated/generation-1'

In [ ]:
!pushd {application.generation_dir}/app_schema
!npm install
!npx tsc
!popd


In [ ]:
typespec = application._make_typespec(application_description)
typespec.score, typespec.data.output

Langfuse client is disabled since no public_key was provided as a parameter or environment variable 'LANGFUSE_PUBLIC_KEY'. See our docs: https://langfuse.com/docs/sdk/python/low-level-sdk#initialize-client


In [ ]:
typescript_schema = application._make_typescript_schema(typespec.data.output.typespec_definitions)
typescript_schema.score, typescript_schema.data.output

In [ ]:
drizzle = application._make_drizzle(typespec.data.output.typespec_definitions)
drizzle.score, drizzle.data.output

In [ ]:
router = application._make_router(typespec.data.output.typespec_definitions)
router.score, router.data.output

In [ ]:
handlers = application._make_handlers(
    typespec.data.output.llm_functions,
    typespec.data.output.typespec_definitions,
    typescript_schema.data.output.typescript_schema,
    drizzle.data.output.drizzle_schema,
)

In [ ]:
for name, handler in handlers.items():
    print(name, handler.score, handler.depth, handler.data.output)
    if handler.score != 1:
        print(handler.data.output.feedback["stdout"])

recordExercise 1 0 HandlerOutput(handler='import { db } from "../db";\nimport type { Exercise } from "../common/schema";\nimport { exerciseTable } from "../db/schema/application";\n\ninterface Options {\n    exercise: Exercise;\n}\n\nexport const handle = async (options: Options): Promise<void> => {\n    await db.insert(exerciseTable).values({\n        name: options.exercise.name,\n        sets: options.exercise.sets,\n        reps: options.exercise.reps,\n        weight: options.exercise.weight,\n        notes: options.exercise.notes\n    }).execute();\n};', feedback={'exit_code': 0, 'stdout': None, 'stderr': None})
updateAvailableEquipment 1 0 HandlerOutput(handler='import { db } from "../db";\nimport { equipmentTable } from "../db/schema/application";\nimport type { Equipment } from "../common/schema";\n\ninterface Options {\n    equipment: Equipment[];\n}\n\nexport const handle = async (options: Options): Promise<void> => {\n    // Clear existing equipment\n    await db.delete(equi